# Exercise 4: Assistants

OpenAI's Assistants API is designed to help developers build powerful AI assistants capable of performing a variety of tasks.

- Assistants can call [models](https://platform.openai.com/docs/models) with specific instructions to tune their personality and capabilities.
- Assistants can access **multiple tools in parallel**. These can be native tools, like `code_interpreter` or `file_search`, or custom tools you build (via function calling).
- Assistants can access persistent **Threads**. Threads simplify AI application development by storing message history and truncating it when the conversation gets too long for the model’s context length. You create a Thread once, and simply append Messages to it as your users reply.
- Assistants can access files in several formats. When using tools, Assistants can also create files (e.g., images, spreadsheets, etc) and reference them in the Messages they create.


In [ ]:
from llm_in_production.openai_utils import get_openai_client

import os
import json
import time
from IPython.display import clear_output
import dotenv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()
# Here we create the client.
client = get_openai_client()


## Overview of Assistants

<p align="center">
<img src="../../assets/diagram-assistant.webp" width=800 align=center />
</p>

| OBJECT     | WHAT IT REPRESENTS                                                                                             |
|------------|---------------------------------------------------------------------------------------------------------------|
| Assistant  | Purpose-built AI that uses OpenAI’s models and calls tools                                                    |
| Thread     | A conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context. |
| Message    | A message created by an Assistant or a user. Messages can include text, images, and other files. Messages stored as a list on the Thread.           |
| Run        | An invocation of an Assistant on a Thread. The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.        |
| Run Step   | A detailed list of steps the Assistant took as part of a Run. An Assistant can call tools or create Messages during its run. Examining Run Steps allows you to introspect how the Assistant is getting to its final results.  |


## Creating Assistants

To get started, creating an Assistant only requires specifying the model to use. But you can further customize the behavior of the Assistant:

- Use the `instructions` parameter to guide the personality of the Assistant and define its goals. Instructions are similar to system messages in the Chat Completions API.
- Use the `tools` parameter to give the Assistant access to native tools, like `code_interpreter` and `file_search`, or call a custom via function calling.
- Use the `tool_resources` parameter to give the tools like `code_interpreter` and `file_search` access to files. Files are uploaded using the `file` upload endpoint and must have the purpose set to assistants to be used with this API.

For example, to create an Assistant that can create data visualization based on a .csv file, first upload a file.

In [ ]:
file = client.files.create(
  file=open("chickweight.csv", "rb"),
  purpose='assistants'
)

Then, create the Assistant, with the `code_interpreter` tool enabled and provide the file as a resource to the tool.

In [ ]:
assistant = client.beta.assistants.create(
  name="Data visualizer",
  description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
  model=os.environ["GPT_4_MODEL_NAME"],
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    },
  }
)

## Managing Threads and Messages

Threads and Messages represent a conversation session between an Assistant and a user. There is no limit to the number of Messages you can store in a Thread. Once the size of the Messages exceeds the context window of the model, the Thread will attempt to smartly truncate messages, before fully dropping the ones it considers the least important.

You can create a Thread with an initial list of Messages like this:

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Create 3 data visualizations based on the trends in this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

Messages can contain text, images, or file attachments. Message `attachments` are helper methods that add files to a thread's `tool_resources`. You can also choose to add files to the `thread.tool_resources` directly.

## Running a thread

You can initate an assistant with a thread like so.

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)
print(run.status)

Depending on the complexity of the query you run, the thread could take longer to execute. In that case you can create a loop to monitor the run status of the thread with code like the example below:

In [ ]:
start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))

When a Run is `in_progress` or in other nonterminal states the thread is locked. When a thread is locked new messages can't be added, and new runs can't be created.

Once the run status indicates successful completion, you can list the contents of the thread again to retrieve the model's and any tools response:

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(messages.model_dump_json(indent=2))

## Retrieving files

We had requested that the model generate three images from the dataset. In order to download the images, we first need to retrieve the images' file IDs.

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object

image_ids = []

for i in range(3):
    image_ids.append(data['data'][0]['content'][i]['image_file']['file_id'])

image_ids

We can then download the images and display them

In [ ]:
fig, axs = plt.subplots(1, len(image_ids), figsize=(12,16))
for i, id in enumerate(image_ids):
    content = client.files.content(id)
    content.write_to_file(f"{i}.png")
    img = mpimg.imread(f"{i}.png")
    axs[i].imshow(img)
    axs[i].axis('off')

plt.show()

## Ask a follow-up question on the thread

We can add follow-up questions if needed.

In [ ]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the graphs"
)

Again we'll need to run the thread and wait for it to complete:

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

In [ ]:
start_time = time.time()
status = run.status
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Status: {status}')

Once the run status reaches completed, we'll list the messages in the thread again which should now include the response to our latest question.

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

To extract only the response to our latest question:

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

## Exercise 4a: Dark-mode

Add another question to the thread to see if the code interpreter can swap the charts to dark mode.

### Part i: Add a user question to the thread

In [ ]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    # YOUR CODE HERE START
    # YOUR CODE HERE END
)


### Part ii: Run the thread and wait for it to complete

In [ ]:
# Run the thread
run = client.beta.threads.runs.create(
  # YOUR CODE HERE START
  # YOUR CODE HERE END
)

In [ ]:
# Check the task has completed
# YOUR CODE HERE START
# YOUR CODE HERE END

print(f'Status: {status}')

### Part iii: Get the Assistants messages

In [ ]:
# Get the Assitant's messages
messages = client.beta.threads.messages.list(
  # YOUR CODE HERE START 
# YOUR CODE HERE END
)

print(messages.model_dump_json(indent=2))

### Visualize the updated graphs

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object

fig, axs = plt.subplots(1, len(image_ids), figsize=(12,16))
for i in range(3):
    id = data['data'][0]['content'][i]['image_file']['file_id']
    content = client.files.content(id)
    content.write_to_file(f"{i}_dark.png")
    img = mpimg.imread(f"{i}_dark.png")
    axs[i].imshow(img)
    axs[i].axis('off')

plt.show()

---